<a href="https://colab.research.google.com/github/ben854719/EstateMind-AI-Your-Real-Estate-AI-Avatar-Assistant/blob/main/Real_Estate_Avatar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade langchain-google-genai google-generativeai
!pip install --upgrade langchain-google-genai google-generativeai langgraph
!pip install langsmith

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 6.2 MB/s eta 0:00:00


In [8]:
import os
os.environ["LANGSMITH_API_KEY"] = "LangSmith"

In [9]:
!pip install "mcp[cli]"
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("GeminiTools")

@mcp.tool()
def search(query: str) -> list:
    # Your search logic here
    return ["Result 1", "Result 2"]

In [10]:
!cd mcp-server-demo
!ls

/bin/bash: line 1: cd: mcp-server-demo: No such file or directory
sample_data


In [11]:
!cd mcp-server-demo && uv add langchain-google-genai langgraph

/bin/bash: line 1: cd: mcp-server-demo: No such file or directory


In [12]:
!pip install requests beautifulsoup4

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from IPython.display import display, HTML
from typing import TypedDict, List
import os
from mcp.server.fastmcp import FastMCP
from google.colab import userdata
from langsmith import traceable
import random
import json
import requests
from bs4 import BeautifulSoup
from langchain_core.tools import Tool

# Import APl Key.
api_key = userdata.get("Ben856")
if not api_key:
    raise ValueError("Ben856 secret not found. Please set your API key in Colab Secrets.")
os.environ["GOOGLE_API_KEY"] = api_key

# Initialize Gemini Model.
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=api_key)
prompt = ChatPromptTemplate.from_template("Respond confidently to: {input}")
chain = prompt | llm

# Dynamic HeyGen Video Selector.
def select_heygen_url(response_text: str) -> str:
    if "onboarding" in response_text.lower():
        return "https://app.heygen.com/videos/your_onboarding_video_id"
    elif "risk" in response_text.lower() or "concern" in response_text.lower():
        return "https://app.heygen.com/videos/your_fallback_video_id"
    else:
        return "https://app.heygen.com/videos/90ce8bbc063746ff915c07052ef1193a"

# Avatar Node (Colab-safe link preview).
def avatar_node(input_text):
    print("Avatar says:", input_text)
    heygen_url = select_heygen_url(input_text)
    display(HTML(f'<a href="{heygen_url}" target="_blank">Click to view avatar response</a>'))
    return {"avatar_output": f"Avatar says: {input_text}"}

# Optional Tool Wrapper.
avatar_tool = Tool.from_function(
    func=avatar_node,
    name="AvatarSpeaker",
    description="Trigger avatar speech"
)

#  Workflow Nodes.
def gemini_node(state: "WorkflowState"):
    response = chain.invoke({"input": state["input"]})
    state["response"] = response.content
    return state

def emotion_node(state: "WorkflowState"):
    text = state["response"]
    if "confident" in text or "welcome" in text:
        state["emotion"] = "positive"
    elif "concern" in text or "risk" in text:
        state["emotion"] = "negative"
    else:
        state["emotion"] = "neutral"
    return state

def fallback_node(state: "WorkflowState"):
    if state["emotion"] == "negative":
        state["response"] = "Fallback message: Let's review the risk indicators together."
    return state

def onboarding_node(state: "WorkflowState"):
    if "onboarding" in state["response"].lower():
        state["response"] = "Welcome aboard! I am bring clarity to our product and able to translate document in English or French based on your  request."
    return state

def compliance_score_node(state: "WorkflowState"):
    score = random.randint(70, 100)
    state["response"] += f" (Compliance Score: {score})"
    return state

def avatar_speak_node(state: "WorkflowState"):
    avatar_result = avatar_node(state["response"])
    state["avatar_output"] = avatar_result["avatar_output"]
    state["input"] = state.get("input", "")  # Ensure input is preserved
    return state


#  State Schema.
class WorkflowState(TypedDict):
    input: str
    response: str
    emotion: str
    avatar_output: str

# Entry node: attach a trace ID for debugging or session tracking
@traceable(name="entry_node")
def entry_node(state: WorkflowState) -> WorkflowState:
    return {
        **state,
        "trace_id": f"trace-{random.randint(1000, 9999)}"
    }

# LLM node: process URLs and generate a HeyGen-ready avatar script.
@traceable(name="llm_node")
def llm_node(state: WorkflowState) -> WorkflowState:
    # Initialize results list
    url_processing_results = state.get("url_processing_results", [])

    # Iterate through target URLs and collect processed content
    for url in state.get("target_urls", []):
        content = select_url(url)  # your custom function
        url_processing_results.append({
            "url": url,
            "content": content
        })

    # Build a HeyGen-friendly avatar script
    avatar_script = build_heygen_script(url_processing_results)

    return {
        **state,
        "url_processing_results": url_processing_results,
        "avatar_script": avatar_script
    }


# Helper: convert processed URLs into a natural, spoken script
def build_heygen_script(results: list) -> str:
    if not results:
        return "I wasn’t able to retrieve any information from the provided links."

    script_lines = ["Here’s a quick overview of what I found:"]

    for item in results:
        url = item["url"]
        content = item["content"]

        # You can customize tone, persona, or style here
        script_lines.append(
            f"From {url}, I found the following key information: {summarize(content)}"
        )

    script_lines.append("Let me know if you'd like a deeper breakdown or a visual summary.")

    return "\n".join(script_lines)


# Simple summarizer placeholder
def summarize(text: str) -> str:
    # Replace with your LLM summarizer or custom logic
    return text[:200] + "..." if text and len(text) > 200 else text

# LangGraph Flow
graph = StateGraph(WorkflowState)
graph.add_node("gemini", gemini_node)
graph.add_node("emotion", emotion_node)
graph.add_node("fallback", fallback_node)
graph.add_node("onboarding", onboarding_node)
graph.add_node("compliance_score", compliance_score_node)
graph.add_node("avatar_speak", avatar_speak_node)

graph.set_entry_point("gemini")
graph.add_edge("gemini", "emotion")
graph.add_edge("emotion", "fallback")
graph.add_edge("fallback", "onboarding")
graph.add_edge("onboarding", "compliance_score")
graph.add_edge("compliance_score", "avatar_speak")
graph.add_edge("avatar_speak", END)

flow = graph.compile()

# Run the Flow.
initial_state = {"input": "Generate a confident greeting for onboarding"}
final_state = flow.invoke(initial_state)

#  Print and Save Final State.
print("Final State:", final_state)

with open("avatar_response.json", "w") as f:
    json.dump(final_state, f, indent=2)

Avatar says: Welcome aboard! I am bring clarity to our product and able to translate document in English or French based on your  request. (Compliance Score: 70)


Final State: {'input': 'Generate a confident greeting for onboarding', 'response': 'Welcome aboard! I am bring clarity to our product and able to translate document in English or French based on your  request. (Compliance Score: 70)', 'emotion': 'positive', 'avatar_output': 'Avatar says: Welcome aboard! I am bring clarity to our product and able to translate document in English or French based on your  request. (Compliance Score: 70)'}
